In [10]:
from tkinter import *
from PIL import ImageTk, Image
from pandastable import Table, TableModel

import nltk
nltk.download('wordnet')

import tweepy,re
from textblob import TextBlob
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from matplotlib import figure
from matplotlib import patches
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot

# authentication
consumerKey = 'CqiCL4TVk8upK7n7mFJeitpiV'
consumerSecret = '7EpkLEYz6WNe0TeGITmJqhGHIh1im1AXWw5sSpopvFUTo9UEXb'
accessToken = '1220024079303892992-cwd1vnIuw9V0q5Rtc3KI5SNUbzGeMw'
accessTokenSecret = 'pcdIdWGvswHCUJasQ08NgjheDmMGu2d9DOvOHEwaeRzs9'
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)


main = Tk()
main.title('Sentiment Analysis')
main.geometry('700x600')
main.config(bg='#1DA1F2') 
main.state('zoomed')

def Clickstart():
    
    labellogo.destroy()
    labeltext.destroy()
    startbtn.destroy()
    canvas.destroy()
    fetch()
    
    
logo = ImageTk.PhotoImage(Image.open('twitter.png'))
labellogo = Label(main,image = logo, bg='#1DA1F2')
labellogo.pack(pady=(50,0))


labeltext = Label(main,text='Sentiment Analysis',font=('Agency FB',68),bg='#1DA1F2')
labeltext.pack(pady=(0, 50))


canvas = Canvas(main, width = 400, height = 100, bg='#1DA1F2', highlightthickness=0)
canvas.pack(pady= (0, 100))

label = Label(main, text='Enter Twitter User ID to Analysis Comments on tweets:')
label.config(font=('helvetica', 10), bg='#1DA1F2')
canvas.create_window(200, 25, window=label)

entry = Entry(main) 
canvas.create_window(200, 65, window=entry)

startbtn = Button(main,
                  font=('calibri',20,'bold'),
                  text='Start',
                  relief='raised',
                  activebackground = '#1DA1F2',
                  border=0,
                  bg='#ffffff',
                  command=Clickstart)

startbtn.pack()

    
def clean_data(text):
    return ' '.join(re.sub("(@[a-zA-Z0-9]+)|([^0-9A-Za-z])|(https://[\w.]+/[\w]+)", " ", text).split())


def drop_numbers(list_text):
    list_text_new = []
    for i in list_text:
        if not re.search('\d', i):
            list_text_new.append(i)
    return ''.join(list_text_new)


# changing all the words of the reviews column to lowercase letters
def lower_case(text):
    text_words = word_tokenize(text)
    text_words_lower = [x.lower() for x in text_words]
    return ' '.join(text_words_lower)


lemmatizer = WordNetLemmatizer()
def lemmatise(text):
    text_tokens = word_tokenize(text)
    text_lemm = [lemmatizer.lemmatize(word) for word in text_tokens]
    return ' '.join(text_lemm)


# Lets calculate the Polarity of the Reviews
def get_polarity(text):
    textblob = TextBlob(str(text))
    pol = textblob.sentiment.polarity
    if(pol==0):
        return "Neutral"
    elif(pol>0 and pol<=0.3):
        return "Weakly Positive"
    elif(pol>0.3 and pol<=0.6):
        return "Positive"
    elif(pol>0.6 and pol<=1):
        return "Strongly Positive"
    elif(pol>-0.3 and pol<=0):
        return "Weakly Negative"
    elif(pol>-0.6 and pol<=-0.3):
        return "Negative"
    elif(pol>-1 and pol<=-0.6):
        return "Strongly Negative"
    
    
def remove_stopword(text):
    text_tokens = word_tokenize(text)
    tokens = [word for word in text_tokens if not word in set(stopwords.words('english'))]
    tokens_text = ' '.join(tokens)
    return tokens_text


def tabledes():
    
    f.destroy()
    pt.destroy()
    plotbtn.destroy()
    
    analysis = Label(
        main,
        text = f'How people are reacting on  {searchTerm.upper()}  by analyzing {str(NoOfTerms)} Tweets',
        font = ('Times New Roman',20),
        justify='center',
        bg='#1DA1F2',
    )
    analysis.pack(pady=(20,10))
    
    
    sizes = [positive, wpositive, spositive, neutral, negative, wnegative, snegative]
    colors = ['yellowgreen','lightgreen','darkgreen', 'gold', 'red','lightsalmon','darkred']
    labels = ['Positive [' + str(positive) + '%]', 'Weakly Positive [' + str(wpositive) + '%]',
              'Strongly Positive [' + str(spositive) + '%]', 'Neutral [' + str(neutral) + '%]',
              'Negative [' + str(negative) + '%]', 'Weakly Negative [' + str(wnegative) + '%]', 
              'Strongly Negative [' + str(snegative) + '%]']

    
    fig = matplotlib.figure.Figure(figsize=(20,15))
    ax = fig.add_subplot(111)

    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', shadow=False)

    circle=matplotlib.patches.Circle( (0,0), 0.7, color='white')
    ax.add_artist(circle)

    insert = FigureCanvasTkAgg(fig,main)
    insert.get_tk_widget().pack()
    
    # main.bind_all('<backbtn>', lambda event: fetchback(insert))
    
    

def fetch():
    
    # input for term to be searched and how many tweets to search
    
    global df, NoOfTerms, searchTerm
    
    searchTerm = entry.get()
    NoOfTerms = 40

    tweets = []
    tweetText = []
    index = []

    # searching for tweets
    tweets = tweepy.Cursor(api.search, q=searchTerm+" -filter:retweets", lang = "en").items(NoOfTerms)

    tweet_list = [tweet.text for tweet in tweets]

    tweet_df = pd.DataFrame(tweet_list)

    tweet_df['Filtered Tweets'] = tweet_df[0].apply(clean_data)

    tweet_df['Filtered Tweets'] = tweet_df['Filtered Tweets'].apply(drop_numbers)

    tweet_df['Filtered Tweets'] = tweet_df['Filtered Tweets'].apply(lower_case)

    tweet_df['Filtered Tweets'] = tweet_df['Filtered Tweets'].apply(lemmatise)

    tweet_df['Filtered Tweets'] = tweet_df['Filtered Tweets'].apply(remove_stopword)

    tweet_df['polarity'] = tweet_df['Filtered Tweets'].apply(get_polarity)

    df = pd.DataFrame(tweet_df)
    
    TestApp()

    global neutral, wpositive, spositive, positive, negative, wnegative, snegative, polarity
    
    neutral = 0
    wpositive = 0
    spositive = 0
    positive = 0
    negative = 0
    wnegative = 0
    snegative = 0
    polarity = 0

    for i in range(0,NoOfTerms):
        textblob = TextBlob(str(df['Filtered Tweets'][i]))
        polarity+= textblob.sentiment.polarity
        pol = textblob.sentiment.polarity
        if (pol == 0):  # adding reaction of how people are reacting to find average later
            neutral += 1
        elif (pol > 0 and pol <= 0.3):
            wpositive += 1
        elif (pol > 0.3 and pol <= 0.6):
            positive += 1
        elif (pol > 0.6 and pol <= 1):
            spositive += 1
        elif (pol > -0.3 and pol <= 0):
            wnegative += 1
        elif (pol > -0.6 and pol <= -0.3):
            negative += 1
        elif (pol > -1 and pol <= -0.6):
            snegative += 1

    # finding average reaction
    polarity = polarity / NoOfTerms

    def percentage(part, whole):
        temp = 100 * float(part) / float(whole)
        return format(temp, '.2f')

    # finding average of how people are reacting
    positive = percentage(positive, NoOfTerms)
    wpositive = percentage(wpositive, NoOfTerms)
    spositive = percentage(spositive, NoOfTerms)
    negative = percentage(negative, NoOfTerms)
    wnegative = percentage(wnegative, NoOfTerms)
    snegative = percentage(snegative, NoOfTerms)
    neutral = percentage(neutral, NoOfTerms) 

    
    
class TestApp(Frame):
        def __init__(self, parent=None):
            
            global pt, f, plotbtn
           
            self.parent = parent
            Frame.__init__(self)
            self.main = self.master
            self.main.geometry('600x500')
            self.main.title('Comments Polarity')
            f = Frame(self.main)
            f.pack(fill=BOTH,expand=1)
            self.table = pt = Table(f, dataframe=df, showtoolbar=True, showstatusbar=True)
            pt.show()
            
            plotbtn = Button(main,
                  text='Plot',
                  font=('calibri',20,'bold'),
                  relief='raised',
                  activebackground = '#1DA1F2',
                  border=0,
                  bg='#ffffff',
                  command= tabledes)
    
            plotbtn.pack()

            return


mainloop()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\racha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\racha\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\racha\AppData\Local\Temp/ipykernel_12212/1627385269.py", line 43, in Clickstart
    fetch()
  File "C:\Users\racha\AppData\Local\Temp/ipykernel_12212/1627385269.py", line 183, in fetch
    tweets = tweepy.Cursor(api.search(), q=searchTerm+" -filter:retweets", lang = "en").items(NoOfTerms)
AttributeError: 'API' object has no attribute 'search'
